In [12]:
import pandas as pd
import numpy as np

# Mostrar floats com duas casas decimas
pd.set_option('display.float_format',  lambda x: '%.2f' % x)
pd.options.display.max_colwidth = 20
pd.options.display.max_columns = 20
pd.options.display.max_rows = 4

In [13]:
# Load stock list
df_magic = pd.read_csv("../data/magic_stocks.csv", parse_dates=["rebalance_on"])
df_magic.rank_final = df_magic.rank_final.astype(int)
df_magic

,rebalance_on,codneg,nomres,codcvm,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield,rank_final
0,2011-04-11,AUTM3,AUTOMETAL,22381,2011-02-28 14:52:34,2010-12-31,94.42,252.11,247.23,1.12,1606.17,1858.28,0.13,1
1,2011-04-11,ETER3,ETERNIT,5762,2011-03-11 11:18:46,2010-12-31,89.50,-56.72,123.66,0.30,987.18,930.47,0.13,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,2022-04-11,BEEF3,MINERVA,20931,2022-02-23 21:43:02,2021-12-31,606.70,6102.70,2035.42,0.22,8251.13,14353.83,0.14,29
359,2022-04-11,JHSF3,JHSF PART,20605,2022-02-24 19:37:23,2021-12-31,686.22,1099.50,1113.51,0.17,4412.42,5511.92,0.20,30


In [14]:
# Slice dataframe with columns that will be used
cols = ['rebalance_on', 'codneg']
df_magic = df_magic.loc[:, cols]
df_magic

,rebalance_on,codneg
0,2011-04-11,AUTM3
1,2011-04-11,ETER3
...,...,...
358,2022-04-11,BEEF3
359,2022-04-11,JHSF3


In [15]:
# List of magic tickers
magic_tickers = list(df_magic['codneg'].unique())
print('Number of tickers =', len(magic_tickers))
print(magic_tickers)

Number of tickers = 114
['AUTM3', 'ETER3', 'TOTS3', 'VALE5', 'RAPT4', 'BRAP4', 'HBOR3', 'FESA4', 'DIRR3', 'TGMA3', 'FHER3', 'CCIM3', 'JHSF3', 'BRKM5', 'UGPA4', 'EVEN3', 'EZTC3', 'ALPA4', 'GRND3', 'MYPK3', 'POMO4', 'NATU3', 'BEEF3', 'FLRY3', 'LREN3', 'AMAR3', 'BISA3', 'BRML3', 'MRVE3', 'POSI3', 'SHOW3', 'ODPV3', 'SLED4', 'MGLU3', 'LAME4', 'DASA3', 'GSHP3', 'RENT3', 'CSAN3', 'PTBL3', 'EMBR3', 'KLBN4', 'TIMP3', 'HGTX3', 'EUCA4', 'BEMA3', 'PCAR4', 'IGTA3', 'RDNI3', 'CCRO3', 'PFRM3', 'LEVE3', 'ARZZ3', 'SEER3', 'CYRE3', 'MAGG3', 'CVCB3', 'ANIM3', 'WEGE3', 'JBSS3', 'ALSC3', 'RLOG3', 'RUMO3', 'FIBR3', 'SUZB5', 'MRFG3', 'BRFS3', 'UGPA3', 'FRAS3', 'SGPS3', 'SMTO3', 'TPIS3', 'MDIA3', 'VULC3', 'CAML3', 'SLCE3', 'PRIO3', 'GUAR3', 'VALE3', 'TEND3', 'MOVI3', 'PARD3', 'JSLG3', 'SMLS3', 'LCAM3', 'SUZB3', 'GOAU4', 'TUPY3', 'CEAB3', 'TRIS3', 'YDUQ3', 'ENAT3', 'LOGN3', 'MTRE3', 'INTB3', 'LAVV3', 'TASA4', 'PLPL3', 'ALLD3', 'CURY3', 'PCAR3', 'USIM5', 'LJQQ3', 'KEPL3', 'RANI3', 'TESA3', 'CMIN3', 'DEXP3', 'GG

In [16]:
# Load complete B3 adjusted price data
# s3://aq-dl/HistoricalQuotations/processed/dataset.feather
# Select magic stocks after 2011 (first year with available accounting data)
# Remove columns that will not be used for backtesting
# Daily average stock price (premed) will be used for entering and exiting positions
file_path = "/mnt/aq_disk/data/HistoricalQuotations/processed/dataset_tr.feather"
df_prices = (pd
    .read_feather(file_path)
    .query('codneg == @magic_tickers and datneg >= "2011.01.01"')
    .reset_index(drop=True)
    [['datneg', 'codneg', 'nomres', 'preult', 'premed']]
)
df_prices

,datneg,codneg,nomres,preult,premed
0,2021-04-12,ALLD3,ALLIED,15.86,16.21
1,2021-04-13,ALLD3,ALLIED,15.79,15.90
...,...,...,...,...,...
240364,2022-07-21,YDUQ3,YDUQS PART,14.09,13.96
240365,2022-07-22,YDUQ3,YDUQS PART,13.90,13.98


In [17]:
# Join price dataframe with magic stocks dataframe
# Since there are stocks in multiple periods, this is a one-to-many join
df_prices = df_prices.merge(right=df_magic, how='inner')
df_prices

,datneg,codneg,nomres,preult,premed,rebalance_on
0,2021-04-12,ALLD3,ALLIED,15.86,16.21,2021-04-12
1,2021-04-12,ALLD3,ALLIED,15.86,16.21,2022-04-11
...,...,...,...,...,...,...
871124,2022-07-21,YDUQ3,YDUQS PART,14.09,13.96,2020-04-09
871125,2022-07-22,YDUQ3,YDUQS PART,13.90,13.98,2020-04-09


In [18]:
# Create a list of ordered unique balancing dates
values = list(df_prices.rebalance_on.sort_values().drop_duplicates())
# Add one extra year to the end of the list
values.append(values[-1] + pd.DateOffset(years=1))
keys = list(range(len(values)))
# Create a dictionary where the keys are the balancing dates
balancing_dict = dict(zip(keys, values))
balancing_dict

{0: Timestamp('2011-04-11 00:00:00'),
 1: Timestamp('2012-04-09 00:00:00'),
 2: Timestamp('2013-04-10 00:00:00'),
 3: Timestamp('2014-04-10 00:00:00'),
 4: Timestamp('2015-04-10 00:00:00'),
 5: Timestamp('2016-04-11 00:00:00'),
 6: Timestamp('2017-04-10 00:00:00'),
 7: Timestamp('2018-04-10 00:00:00'),
 8: Timestamp('2019-04-10 00:00:00'),
 9: Timestamp('2020-04-09 00:00:00'),
 10: Timestamp('2021-04-12 00:00:00'),
 11: Timestamp('2022-04-11 00:00:00'),
 12: Timestamp('2023-04-11 00:00:00')}

In [19]:
# Rank the balancing dates so we have the keys for mapping with the balancing_dict
df_prices['balance_key'] = df_prices['rebalance_on'].rank(method='dense').astype(int)
df_prices

,datneg,codneg,nomres,preult,premed,rebalance_on,balance_key
0,2021-04-12,ALLD3,ALLIED,15.86,16.21,2021-04-12,11
1,2021-04-12,ALLD3,ALLIED,15.86,16.21,2022-04-11,12
...,...,...,...,...,...,...,...
871124,2022-07-21,YDUQ3,YDUQS PART,14.09,13.96,2020-04-09,10
871125,2022-07-22,YDUQ3,YDUQS PART,13.90,13.98,2020-04-09,10


In [20]:
# Map values so that we have the next year balancing dates
df_prices['next_rebalance'] = df_prices['balance_key'].map(balancing_dict)
df_prices.drop(columns='balance_key', inplace=True)
df_prices

,datneg,codneg,nomres,preult,premed,rebalance_on,next_rebalance
0,2021-04-12,ALLD3,ALLIED,15.86,16.21,2021-04-12,2022-04-11
1,2021-04-12,ALLD3,ALLIED,15.86,16.21,2022-04-11,2023-04-11
...,...,...,...,...,...,...,...
871124,2022-07-21,YDUQ3,YDUQS PART,14.09,13.96,2020-04-09,2021-04-12
871125,2022-07-22,YDUQ3,YDUQS PART,13.90,13.98,2020-04-09,2021-04-12


In [21]:
# Select prices that are between each of the balancing intervals
df_prices.query('rebalance_on <= datneg <= next_rebalance', inplace=True)
df_prices.sort_values(['codneg', 'datneg'], inplace=True, ignore_index=True)
df_prices

,datneg,codneg,nomres,preult,premed,rebalance_on,next_rebalance
0,2021-04-12,ALLD3,ALLIED,15.86,16.21,2021-04-12,2022-04-11
1,2021-04-13,ALLD3,ALLIED,15.79,15.90,2021-04-12,2022-04-11
...,...,...,...,...,...,...,...
83135,2021-04-09,YDUQ3,YDUQS PART,30.57,30.53,2020-04-09,2021-04-12
83136,2021-04-12,YDUQ3,YDUQS PART,30.58,30.59,2020-04-09,2021-04-12


In [22]:
df_prices.to_csv('../data/adjusted_prices.csv', index=False)